# wat is de connectie tussen vertragingen en het weer.

in dit notebook ga ik proberen te achterhallen of het weer een roll speelt met een vertraging in het openbaren trein verkeer.



met deze datasets hoop ik te kunnen vinden of en waar er een goede verbinding is.


in mijn chalage ga ik gebruik maken van phyton pandas, een library om datasets in te laden. ik ga daarnaast ook gebruik maken van plotly om het mooi aan te geven en sklearn voor de machine learning algoritmes.

dependencies voor de chalange:

In [29]:
# dependacies
import os
import numpy as np
# import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
# from geopy.exc import GeocoderTimedOut
# from geopy.geocoders import Nominatim
# from geopy.distance import geodesic
import plotly.express as px
import plotly.graph_objects as go

from sklearn.impute import KNNImputer

datafolder = 'data/'

max_data = datetime(2023, 3, 31, 23, 59, 59)

# 📦 Data opzet

nu ben ik op zoek gegaan naar datasets die van belang kunnen zijn in mijn opdracht. 

dit zijn data sets die ik heb gekozen:
- [rijden de trijnen. Treinarchief](https://www.rijdendetreinen.nl/open-data/treinarchief)
- [rijden de trijnen. treinstoringen](https://www.rijdendetreinen.nl/open-data/treinstoringen)
- [KNMI weerstations Uurwaarnemingen](https://www.daggegevens.knmi.nl/klimatologie/uurgegevens)
- [Train Stations in Europe](https://www.kaggle.com/datasets/headsortails/train-stations-in-europe)


in deze datasets staat het all het trein verkeer, het trein verkeer dat van gezecht word dat het een storing was, het weer in uren en de locatie van het trein station.

### treinarchief laden.

de code hieronder laad het trainarchief in. in dit treinarchief hebben we de volgende colomen volgens [rijden de trijnen](https://www.rijdendetreinen.nl/):
- `Service:RDT-ID` Dit ID wordt gebruikt voor deze rit. Het heeft geen nuttige betekenis buiten het uniek identificeren van een enkele rit op een enkele datum.
>Dit ID zal meerdere keren voorkomen in de CSV-bestanden omdat dit ID uniek is voor iedere rit. Er is ook een kolom Stop:RDT-ID met een uniek ID voor iedere stop.
- `Service:Date` De geplande datum voor deze rit.
>De ritdatum is niet altijd de logische datum. Bijvoorbeeld: een rit die vertrekt om 23:59 op 31 juli en aankomt om 02:00 op 1 augustus heeft een ritdatum van 31 juli. Vertragingen hebben geen invloed op de ritdatum.
- `Service:Type` Het soort trein, zoals Intercity, Sprinter of ICE International.
- `Service:Company` Vervoerder die deze rit uitvoert, zoals NS of Arriva.
- `Service:Train number` Het treinnummer (ritnummer) voor deze rit identificeert deze rit uniek op deze datum. Dit nummer wordt soms ook gecommuniceerd naar reizigers (vooral voor internationale treinen). 
> Een enkele rit kan soms meerdere treinnummers hebben. Bijvoorbeeld wanneer een trein gesplitst wordt in twee delen, of wanneer een trein op een groot station halverwege van treinnummer verandert.
- `Service:Completely cancelled` Deze kolom is true wanneer alle stops van deze rit opgeheven zijn. Of met andere woorden: wanneer de trein helemaal niet rijdt.
- `Service:Partly cancelled` Deze kolom is true wanneer een of meer stops van deze rit opgeheven zijn. Of met andere woorden: wanneer de trein niet rijdt op een deel van het traject.
- `Service:Maximum delay` De hoogste vertraging (in minuten) van alle stops van deze rit.
- `Stop:RDT-ID` Unieke ID voor deze stop. Dit ID is uniek voor iedere stop in de dataset. Het heeft verder geen nuttige betekenis.
- `Stop:Station code` Code (verkorting) van de stationsnaam.
- `Stop:Station name` De naam van het station.
- `Stop:Arrival time` Geplande aankomsttijd in RFC 3339-formaat. Deze kolom is leeg wanneer er geen aankomst gepland was.
- `Stop:Arrival delay` Vertraging bij aankomst in minuten. Deze kolom is leeg wanneer er geen aankomst gepland was.
- `Stop:Arrival cancelled` Deze kolom is true wanneer de aankomst op deze stop opgeheven is. Deze kolom is leeg wanneer er geen aankomst gepland was.
- `Stop:Departure time` Geplande vertrektijd in RFC 3339-formaat. Deze kolom is leeg wanneer er geen vertrek gepland was.
- `Stop:Departure delay` Vertraging bij vertrek in minuten. Deze kolom is leeg wanneer er geen vertrek gepland was.
- `Stop:Departure cancelled` Deze kolom is true wanneer het vertrek op deze stop opgeheven is. Deze kolom is leeg wanneer er geen vertrek gepland was.


In de code laad ik de dataset in en verander ik de datum naar een datetime zodat ik hem dan kan verminderen naar het zoekgebied die ik op het moment heb in de weer dataset. daarna zet ik type en bedrijf naar kleine letters omdat de dataset handmatig soms is aangepast en dan worden hoofdletter vergeten.

In [30]:
Train_trafic_2023 = pd.read_csv(datafolder + 'services-2023/services-2023.csv', sep=',', low_memory=False)

Train_trafic_2023['Service:Date'] = pd.to_datetime(Train_trafic_2023['Service:Date'], format="%Y-%m-%d")

Train_trafic_2023['Service:Type'] = Train_trafic_2023['Service:Type'].str.lower()
Train_trafic_2023['Service:Company'] = Train_trafic_2023['Service:Company'].str.lower()

# Train_trafic_2023 = Train_trafic_2023[Train_trafic_2023['Service:Company'] == 'ns']

Train_trafic_2023 = Train_trafic_2023[Train_trafic_2023['Service:Date'] < max_data]

print(f'er zijn {len(Train_trafic_2023)} entries ingeladen') 

er zijn 5142250 entries ingeladen


### KNMI Uurwaarnemingen laden

in de code hieronder laad ik de uurwaarneming van het weer in. dit is waarom meerendeels van het project alleen over de eerse 2 maanden van het project gaan. <p>
in de dataset van het weer hebben we:
- `YYYYMMDD` datum (YYYY=jaar;MM=maand;DD=dag)
- `HH` tijd (HH=uur; UT.12 UT=13 MET; 14 MEZT. Uurvak 05 loopt van 04.00 UT tot 5.00 UT
- `DD` Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord; 90=oost; 180=zuid; 270=west; 0=windstil 990=veranderlijk. Zie [Hier](http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken) / Mean wind direction (in degrees) during the 10-minute period preceding the time of observation (360=north; 90=east; 180=south; 270=west; 0=calm 990=variable)
- `FH` Uurgemiddelde windsnelheid (in 0.1 m/s). Zie [Hier](http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken) / Hourly mean wind speed (in 0.1 m/s)
- `FF` Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur / Mean wind speed (in 0.1 m/s) during the 10-minute period preceding the time of observation
- `FX` Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak / Maximum wind gust (in 0.1 m/s) during the hourly division
- `T` Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming / Temperature (in 0.1 degrees Celsius) at 1.50 m at the time of observation
- `T10N` Minimumtemperatuur (in 0.1 graden Celsius) op 10 cm hoogte in de afgelopen 6 uur / Minimum temperature (in 0.1 degrees Celsius) at 0.1 m in the preceding 6-hour period
- `TD` Dauwpuntstemperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming / Dew point temperature (in 0.1 degrees Celsius) at 1.50 m at the time of observation
- `SQ` Duur van de zonneschijn (in 0.1 uren) per uurvak; berekend uit globale straling  (-1 for <0.05 uur) / Sunshine duration (in 0.1 hour) during the hourly division; calculated from global radiation (-1 for <0.05 hour)
- `Q` Globale straling (in J/cm2) per uurvak / Global radiation (in J/cm2) during the hourly division
- `DR` Duur van de neerslag (in 0.1 uur) per uurvak / Precipitation duration (in 0.1 hour) during the hourly division
- `RH` Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm) / Hourly precipitation amount (in 0.1 mm) (-1 for <0.05 mm)
- `P` Luchtdruk (in 0.1 hPa) herleid naar zeeniveau; tijdens de waarneming / Air pressure (in 0.1 hPa) reduced to mean sea level; at the time of observation
- `VV` Horizontaal zicht tijdens de waarneming (0=minder dan 100m; 1=100-200m; 2=200-300m;...; 49=4900-5000m; 50=5-6km; 56=6-7km; 57=7-8km; ...; 79=29-30km; 80=30-35km; 81=35-40km;...; 89=meer dan 70km) / Horizontal visibility at the time of observation (0=less than 100m; 1=100-200m; 2=200-300m;...; 49=4900-5000m; 50=5-6km; 56=6-7km; 57=7-8km; ...; 79=29-30km; 80=30-35km; 81=35-40km;...; 89=more than 70km)
- `N` Bewolking (bedekkingsgraad van de bovenlucht in achtsten); tijdens de waarneming (9=bovenlucht onzichtbaar) / Cloud cover (in octants); at the time of observation (9=sky invisible)
- `U` Relatieve vochtigheid (in procenten) op 1.50 m hoogte tijdens de waarneming / Relative atmospheric humidity (in percents) at 1.50 m at the time of observation
- `WW` Weercode (00-99); visueel(WW) of automatisch(WaWa) waargenomen; voor het actuele weer of het weer in het afgelopen uur. Zie [Hier](http://bibliotheek.knmi.nl/scholierenpdf/weercodes_Nederland) / Present weather code (00-99); description for the hourly division.
- `IX` Weercode indicator voor de wijze van waarnemen op een bemand of automatisch station (1=bemand gebruikmakend van code uit visuele waarnemingen; 2;3=bemand en weggelaten (geen belangrijk weersverschijnsel; geen gegevens); 4=automatisch en opgenomen (gebruikmakend van code uit visuele waarnemingen); 5;6=automatisch en weggelaten (geen belangrijk weersverschijnsel; geen gegevens); 7=automatisch gebruikmakend van code uit automatische waarnemingen) / Indicator present weather code (1=manned and recorded (using code from visual observations); 2;3=manned and omitted (no significant weather phenomenon to report; not available); 4=automatically recorded (using code from visual observations); 5;6=automatically omitted (no significant weather phenomenon to report; not available); 7=automatically set (using code from automated observations)
- `M` Mist 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Fog 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
- `R` Regen 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Rainfall 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
- `S` Sneeuw 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Snow 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
- `O` Onweer 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Thunder  0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
- `Y` IJsvorming 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Ice formation 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation


hier laad ik het weer in dat en zet de start_time of de start tijd naar een datetime. daarby combineer ik twee colommen om een datum en tijd in 1 colom te krijgen.

In [31]:
weather_2023 = pd.read_csv(datafolder + '2023-jan-mar.csv', sep=',', low_memory=False, encoding='utf-8', skiprows=80)

weather_2023['start_time'] = weather_2023['start_time'].astype(str) + ' ' + (weather_2023['HH'] - 1).astype(str) + ':00:00'

weather_2023['start_time'] = pd.to_datetime(weather_2023['start_time'], format="%Y%m%d %H:%M:%S")
#  drop columns that are not needed
# weather_2023 = weather_2023.drop(columns=['HH', 'WW', 'IX', 'FH', 'FX', 'TD', 'SQ', 'RH'])
weather_2023 = weather_2023.drop(columns=['HH'])

print(f'er zijn {len(weather_2023)} entries ingeladen') 

er zijn 99360 entries ingeladen


### trein station 

in de dataset hieronder zitten alle trein stations van europa. ik zal minder nodig hebben dus zal met de country alleen nederland, duitserland en belgie gaan houden. dit is ook omdat mijn andere datasets niet verder kijken dan die locaties.

in deze dataset zit:
- `id` 
- `name` 
- `name_norm` 
- `uic` 
- `latitude` 
- `longitude` 
- `country` 
- `time_zone` 
- `is_city` 
- `is_main_station` 
- `is_airport` 
- `entur_id` 
- `entur_is_enabled` 

ik zal deze dataset gebruiken om de locaties van de trein stations te vinden. <p>
en aangezien ik alleen de locaties hoef te hebben zal ik die ook allen onthouden. naast de belangerijke data zoals Namen die het kunnen zijn en in welk land ze zitten.



In [32]:
europa_train_station_locations = pd.read_csv(datafolder + 'train_stations_europe.csv', low_memory=False, encoding='utf-8')

europa_train_station_locations = europa_train_station_locations.drop(columns=['id', 'uic', 'parent_station_id', 'time_zone', 'is_city', 'is_main_station', 'is_airport', 'entur_id', 'entur_is_enabled'])

europa_train_station_locations['country'] = pd.Categorical(europa_train_station_locations['country'], categories=['NL', 'DE', 'BE'], ordered=True)
europa_train_station_locations = europa_train_station_locations[europa_train_station_locations['country'].notna()]

print(f'er zijn {len(europa_train_station_locations)} entries ingeladen') 

er zijn 8934 entries ingeladen


# 📃 Sample de data

om even naar de data te kijken ga ik 20 van iedere dataset hieronder laden. dit is om te kijken of het wel goed is ingeladen en dat de data er een beetje werkzaam uitziet.

### treinarchief laden.

hier laad ik de samples van het trein verkeer.

In [53]:
sample_size = 20

display(Train_trafic_2023.sample(sample_size))

,Service:RDT-ID,Service:Date,Service:Type,Service:Company,Service:Train number,Service:Completely cancelled,Service:Partly cancelled,Service:Maximum delay,Stop:RDT-ID,Stop:Station code,Stop:Station name,Stop:Arrival time,Stop:Arrival delay,Stop:Arrival cancelled,Stop:Departure time,Stop:Departure delay,Stop:Departure cancelled
582264,10427236,2023-01-11,sprinter,ns,6075,False,False,0,93655068,CL,Culemborg,2023-01-11T20:59:00+01:00,0.0,False,2023-01-11T20:59:00+01:00,0.0,False
1913079,10568042,2023-02-03,stoptrein,arriva,31168,False,False,0,94985874,KTR,Kesteren,2023-02-03T21:04:00+01:00,0.0,False,2023-02-03T21:04:00+01:00,0.0,False
4964137,10897634,2023-03-29,stoptrein,valleilijn,31312,False,False,0,98036775,BNN,Barneveld Noord,2023-03-29T08:46:00+02:00,0.0,False,2023-03-29T08:47:00+02:00,0.0,False
748262,10444541,2023-01-14,sprinter,ns,4860,False,False,1,93820897,UTG,Uitgeest,NaN,NaN,NaN,2023-01-14T19:21:00+01:00,0.0,False
1768155,10553315,2023-02-01,sprinter,ns,6353,False,False,0,94840944,HLM,Haarlem,NaN,NaN,NaN,2023-02-01T13:25:00+01:00,0.0,False
4888434,10889809,2023-03-26,intercity,ns,703750,False,True,0,97961038,DN,Deurne,2023-03-26T18:35:00+02:00,3.0,False,2023-03-26T18:35:00+02:00,3.0,False
602200,10429284,2023-01-12,intercity,ns,3628,False,False,0,93675000,NM,Nijmegen,2023-01-12T09:52:00+01:00,0.0,False,2023-01-12T09:54:00+01:00,0.0,False
4445166,10842069,2023-03-20,sprinter,blauwnet,7941,False,False,0,97517969,ES,Enschede,2023-03-20T14:26:00+01:00,0.0,False,NaN,NaN,NaN
2067252,10585982,2023-02-06,stoptrein,arriva,20158,False,False,0,95140070,GERP,Groningen Europapark,2023-02-06T19:20:00+01:00,2.0,False,2023-02-06T19:20:00+01:00,2.0,False
3878603,10782871,2023-03-10,intercity,ns,1831,False,False,0,96951420,ASDZ,Amsterdam Zuid,2023-03-10T09:41:00+01:00,2.0,False,2023-03-10T09:42:00+01:00,2.0,False


### KNMI Uurwaarnemingen laden
hieronder laad ik de samples can de weerstations.

In [54]:
display(weather_2023.sample(sample_size))

,STN,start_time,DD,FH,FF,FX,temp,T10N,TD,SQ,...,VV,N,U,WW,IX,M,R,S,O,Y
56497,310,2023-01-15 01:00:00,250.0,140.0,140.0,180.0,82.0,NaN,27.0,0.0,...,65.0,8.0,68.0,NaN,5,0.0,0.0,0.0,0.0,0.0
31080,270,2023-02-05 00:00:00,200.0,30.0,30.0,60.0,62.0,NaN,56.0,0.0,...,57.0,8.0,96.0,23.0,7,0.0,1.0,0.0,0.0,0.0
50132,286,2023-01-19 20:00:00,220.0,40.0,40.0,50.0,3.0,NaN,-13.0,0.0,...,NaN,NaN,89.0,NaN,6,NaN,NaN,NaN,NaN,NaN
10890,242,2023-01-04 18:00:00,250.0,140.0,140.0,180.0,108.0,NaN,83.0,NaN,...,57.0,8.0,84.0,23.0,7,0.0,1.0,0.0,0.0,0.0
73648,330,2023-01-09 16:00:00,300.0,100.0,100.0,140.0,80.0,NaN,47.0,0.0,...,NaN,NaN,79.0,NaN,6,NaN,NaN,NaN,NaN,NaN
93979,377,2023-02-15 19:00:00,250.0,30.0,30.0,50.0,70.0,NaN,31.0,0.0,...,64.0,8.0,75.0,NaN,5,0.0,0.0,0.0,0.0,0.0
67796,319,2023-02-04 20:00:00,260.0,30.0,30.0,50.0,79.0,NaN,72.0,0.0,...,61.0,NaN,95.0,NaN,5,0.0,0.0,0.0,0.0,0.0
27236,267,2023-02-24 20:00:00,330.0,120.0,110.0,190.0,52.0,NaN,14.0,0.0,...,70.0,6.0,76.0,23.0,7,0.0,1.0,0.0,0.0,0.0
42993,279,2023-03-23 09:00:00,230.0,80.0,100.0,140.0,130.0,NaN,88.0,6.0,...,75.0,5.0,75.0,22.0,7,0.0,1.0,0.0,0.0,0.0
14299,248,2023-02-25 19:00:00,10.0,40.0,40.0,60.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN


### trein station 
hier laad ik de samples van de trein stations

In [55]:
display(europa_train_station_locations.sample(sample_size))

,name,name_norm,latitude,longitude,country
5920,Bad Kreuznach,Bad Kreuznach,49.842399,7.866433,DE
36291,Eibau Bahnhofstraße,Eibau Bahnhofstrasse,50.987534,14.652255,DE
10360,Immendingen Zimmern,Immendingen Zimmern,47.932874,8.714053,DE
13125,Kahla (Thür),Kahla (Thur),50.808964,11.588947,DE
13118,Chemnitz Erfenschlag,Chemnitz Erfenschlag,50.787533,12.950138,DE
26884,Achterwehr,Achterwehr,54.314910,9.964021,DE
9499,Dahlem (Eifel),Dahlem (Eifel),50.385752,6.550385,DE
5504,Münchberg,Munchberg,50.189041,11.790665,DE
12533,Potsdam Charlottenhof,Potsdam Charlottenhof,52.392800,13.036498,DE
14652,Strausberg Stadt,Strausberg Stadt,52.576980,13.888136,DE
